# 2.5D DC Resistivity Inversion

This tutorial demonstrates DC resistivity inversion with SimPEG. We load normalized voltage data and 2D topography for a synthetic pole-dipole survey. Once loaded, we demonstrate a standard set of steps for recovering a model that characterizing subsurface electrical resistivity using weighted least-squares inversion.

The following items are emphasized in this tutorial:

- Assigning uncertainties to DC resistivity data.
- Designing a suitable mesh based on survey geometry.
- What model parameters we should invert for.
- Defining the inverse problem (data misfit, regularization, optimization).
- Stopping criteria for the inversion.
- Assessing inversion outputs.

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/dcr_inv.png" width="100%" align="center"/>

## Step 0: Importing Modules

In [ ]:
# SimPEG functionality
from SimPEG.electromagnetics.static import resistivity as dc
from SimPEG.electromagnetics.static.utils.static_utils import (
    plot_pseudosection,
    apparent_resistivity_from_voltage,
    pseudo_locations,
)
from SimPEG.utils.io_utils.io_utils_electromagnetics import read_dcip2d_ubc
from SimPEG.utils import model_builder, Counter
from SimPEG import (
    maps,
    data_misfit,
    regularization,
    optimization,
    inverse_problem,
    inversion,
    directives,
)

# discretize functionality
from discretize import TreeMesh
from discretize.utils import active_from_xyz

# Basic Python functionality
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# Pooch for downloading files from the web
import pooch

mpl.rcParams.update({"font.size": 14})  # default font size
cmap = mpl.cm.RdYlBu_r  # default colormap

## Step1: Load Data and Plot

### Download the Data

In [ ]:
topo_url = "https://github.com/simpeg/agrogeo24/raw/main/data/topo_2d.txt"
topo_hash = "B331A4C8D800814A226BB50F9AF76BE08EF439E8221CBA559A1DCCB70C517E2C"

topo_filename = pooch.retrieve(topo_url, known_hash=topo_hash)

In [ ]:
data_url = "https://github.com/simpeg/agrogeo24/raw/main/data/dc_data.obs"
data_hash = "BDB4C9A88A2955468E28994E62B99C9549417822456CA72CBF7591AE7327553A"

data_filename = pooch.retrieve(data_url, known_hash=data_hash)

### Load Topography

In [ ]:
topo_2d = np.loadtxt(str(topo_filename))
print(topo_2d)

### Load the Observed Data

In [ ]:
dc_data = read_dcip2d_ubc(data_filename, "volt", "general")

### Plot the Topography and Electrode Locations

In [ ]:
unique_locations = dc_data.survey.unique_electrode_locations

fig = plt.figure(figsize=(10, 2))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])
ax.plot(topo_2d[:, 0], topo_2d[:, -1], color="b", linewidth=1)
ax.plot(unique_locations[:, 0], unique_locations[:, -1], "ro", markersize=4)
ax.set_xlim([topo_2d[:, 0].min(), topo_2d[:, 0].max()])
ax.set_xlabel("x (m)", labelpad=5)
ax.set_ylabel("y (m)", labelpad=5)
ax.grid(True)
ax.set_title("Topography and Electrode Locations", fontsize=16, pad=10)
plt.show(fig)

### Plot Observed Data in Pseudo-Section

In [ ]:
# Plot voltages pseudo-section
fig = plt.figure(figsize=(8, 2.75))
ax1 = fig.add_axes([0.1, 0.15, 0.75, 0.78])
plot_pseudosection(
    dc_data,
    plot_type="scatter",
    ax=ax1,
    scale="log",
    cbar_label="V/A",
    scatter_opts={"cmap": mpl.cm.viridis},
)
ax1.set_title("Normalized Voltages")
plt.show()

# Get apparent conductivities from volts and survey geometry
apparent_resistivities = apparent_resistivity_from_voltage(dc_data.survey, dc_data.dobs)

# Plot apparent resistivity pseudo-section
fig = plt.figure(figsize=(8, 2.75))
ax1 = fig.add_axes([0.1, 0.15, 0.75, 0.78])
plot_pseudosection(
    dc_data.survey,
    apparent_resistivities,
    plot_type="contourf",
    data_locations=True,
    ax=ax1,
    scale="log",
    cbar_label="$\Omega m$",
    mask_topography=True,
    contourf_opts={"levels": 20, "cmap": mpl.cm.RdYlBu_r},
)
ax1.set_title("Apparent Resistivity")
plt.show()

## Step 2: Assign Uncertainties 

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/true_observed_data.png" width="90%" align="center"/>

### <span style="color:darkgreen">Exercise (beginner):</span>

Uncertainties are estimates of the standard deviations of the noise on our data. Assign uncertainties of 1e-4 V/A + 8 % to all data. We can do this by setting the *standard_deviation* property of our *data object*.

In [ ]:
inds_sort = np.argsort(np.abs(dc_data.dobs))
sorted_data = np.abs(dc_data.dobs[inds_sort])
sorted_std = dc_data.standard_deviation[inds_sort]

fig = plt.figure(figsize=(7, 5))
ax = fig.add_axes([0.1, 0.1, 0.8, 0.8])

x_plot = np.arange(0, len(sorted_data))
ax.semilogy(x_plot, sorted_data, "k.", markersize=1)

ax.set_title("Sorted Data and Uncertainties")
ax.fill_between(x_plot, sorted_data - sorted_std, sorted_data + sorted_std, alpha=0.5)
ax.grid(alpha=0.5)
ax.set_ylabel("Observed Data (V/A)")

## Step 3: Designing a (Tree) Mesh

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/tree_mesh.png" width="50%" align="center"/>

### <span style="color:darkgreen">Exercise (beginner):</span>

Determine the minimum cell size *dh* and the domain widths *dom_width_x* and *dom_width_y* from the survey geometry.

1. Obtain the unique electrode location from the *dc_data* object
2. Use this to find the minimum and maximum electrode spacing
3. Use this to determine good values for *dh*, *dom_width_x* and *dom_width_y*

In [ ]:
nbcx = 2 ** int(np.round(np.log(dom_width_x / dh) / np.log(2.0)))  # num. base cells x
nbcy = 2 ** int(np.round(np.log(dom_width_y / dh) / np.log(2.0)))  # num. base cells y

# Define the base mesh with top at z = 0 m
hx = [(dh, nbcx)]
hy = [(dh, nbcy)]
mesh = TreeMesh([hx, hy], x0="CN")

# Shift top to maximum topography and center of survey line
mesh.origin = mesh.origin + np.r_[np.median(topo_2d[:, 0]), topo_2d[:, -1].max()]

In [ ]:
# Mesh refinement based on topography
mesh.refine_surface(
    topo_2d,
    padding_cells_by_level=[0, 0, 4, 4],
    finalize=False,
)

# Extract unique electrode locations.
unique_locations = dc_data.survey.unique_electrode_locations

# Mesh refinement near electrodes.
mesh.refine_points(
    unique_locations, padding_cells_by_level=[8, 12, 6, 6], finalize=False
)

In [ ]:
# Finalize the mesh
mesh.finalize()

## Step 4: Define the Active Cells

Use the [active_from_xyz](https://discretize.simpeg.xyz/en/main/api/generated/discretize.utils.active_from_xyz.html) utility function to find the indices of the active cells using the mesh and surface topography. The output quantity is a ``bool`` array.

In [ ]:
active_cells = active_from_xyz(mesh, topo_2d)

## Step 5: Project Survey to Discretized Topography

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/project_electrodes.png" width="30%" align="center"/>

Use the [drape_electrodes_on_topography](https://docs.simpeg.xyz/content/api/generated/SimPEG.electromagnetics.static.resistivity.Survey.drape_electrodes_on_topography.html#SimPEG.electromagnetics.static.resistivity.Survey.drape_electrodes_on_topography) method to project electrodes to the discrete surface topography.

In [ ]:
dc_data.survey.drape_electrodes_on_topography(mesh, active_cells, option="top")

## Step 6: Mapping from the Model to the Mesh

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/mapping_2.png" width="80%" align="center"/>

### <span style="color:darkgreen">Exercise (beginner):</span>

1. Generate an [SimPEG.maps.ExpMap](https://docs.simpeg.xyz/content/api/generated/SimPEG.maps.ExpMap.html#SimPEG.maps.ExpMap) mapping object to map log-resistivities to resistivities.
2. Generate a [SimPEG.maps.InjectActiveCells](https://docs.simpeg.xyz/content/api/generated/SimPEG.maps.InjectActiveCells.html#SimPEG.maps.InjectActiveCells) mapping object to project values on active cells to the entire mesh. Make sure to fix the value of inactive (air) cells to 1e8 $\Omega m$.
3. Use the $*$ operator to combine the separate mapping objects into a single mapping.

## Step 7: Define the Forward Simulation

In [ ]:
dc_simulation = dc.simulation_2d.Simulation2DNodal(
    mesh, survey=dc_data.survey, rhoMap=log_resistivity_map, storeJ=True
)

## Step 8: Starting/Reference Models

The **starting model** defines a reasonable starting point for the inversion.

The **reference model** is used to include a-priori geological information in the recovered model.

### <span style="color:darkgreen">Exercise (beginner):</span>

Use the median apparent resistivity value to make a halfspace starting model.

## Step 9: Define the Inverse Problem

The solution to the inverse problem is the model $\mathbf{m}$ which minimizes an objective (penalty) function $\phi (\mathbf{m})$ of the form:

$$
\min_{\mathbf{m}} \; \phi (\mathbf{m}) = \phi_d (\mathbf{m}) + \beta \, \phi_m (\mathbf{m})\\
$$

where

* $\phi_d (\mathbf{m})$ is the data misfit. It is a measure of how well data predicted with a model $\mathbf{m}$ fits the data.
* $\phi_m (\mathbf{m})$ is the regularization. It is where we impose structural constraints on the recovered model.
* $\beta$ is the trade-off parameter that balances the relative emphasis of fitting the data and imposing structure.

### Step 9a: Define the Data Misfit

The data misfit $\phi_d (\mathbf{m})$ is a measure of how well data predicted with a given model $\mathbf{m}$ fits the observed data. Here, the data misfit is defined as the L2-norm of a weighted residual:

$$
\phi_d (\mathbf{m})
= \sum_{i=1}^{nD} \, \Bigg ( \frac{d_i^{pred} - d_i^{obs}}{\varepsilon_i} \Bigg )^2
= \big \| \mathbf{W_d} \big ( F[\mathbf{m}] - \mathbf{d^{obs}} \big ) \big \|^2
$$

We define $F[\mathbf{m}]$ as the predicted data for a given model. And $\mathbf{W_d}$ is a diagonal matrix that weights the residual by the data uncertainties.

### <span style="color:darkgreen">Exercise (beginner):</span>

Generate a [data_misfit.L2DataMisfit](https://docs.simpeg.xyz/content/api/generated/SimPEG.data_misfit.L2DataMisfit.html) object. You must provide the *simulation* and *data* objects.

### Step 9b: Regularization

The regularization function $\phi_m (\mathbf{m})$ is where we impose structural constraints on the recovered model. Here, we use a weighted least-squares regularization:

$$
\phi_m (\mathbf{m})
= \alpha_s \, \big \| \mathbf{W_s} \big ( \mathbf{m - m}_{ref} \big ) \big \|^2
+ \alpha_x \, \big \| \mathbf{W_x \, G_x \, m} \big \|^2
+ \alpha_y \, \big \| \mathbf{W_z \, G_y \, m} \big \|^2
$$

where 

* $\alpha_s, \alpha_x, \alpha_y$ balance the relative emphasis on each term,
* $\mathbf{W}_s, \mathbf{W}_x, \mathbf{W}_y$ are matrices that apply cell weights,
* $\mathbf{G_x}$ and $\mathbf{G_y}$ are partial gradient operators,
* and $\mathbf{m}_{ref}$ is a user-defined reference model

### <span style="color:darkgreen">Exercise (beginner):</span>

Generate a [regularization.WeightedLeastSquares](myst:SimPEG#SimPEG.regularization.WeightedLeastSquares) object. For this, we require the:

* mesh
* active cells
* user-specified parameters

We are going use kwargs to set *length_scale_x* and *length_scale_y* to 10.

### <span style="color:darkorange">Exercise (advanced):</span>

Try difference length scales or introduce a reference model that has structure.

### Step 9c: Optimization

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/newton_iteration.png" width="30%" align="right"/>

For a given trade-off parameter $\beta$, we must solve the following optimization problem:

$$
\min_{\mathbf{m}} \; \phi (\mathbf{m}) = \phi_d (\mathbf{m}) + \beta \phi_m (\mathbf{m}) \\
$$

Because our data misfit and regularization functions are parabolic, the solution $\mathbf{m^*}$ occurs when the gradient is equal to zero, i.e.:

$$
\nabla \phi(\mathbf{m^*}) = \mathbf{0}
$$

In practice, this problem is solved iteratively using quasi-Newton methods (see figure). Where $\mathbf{H}$ represents an approximation of the Hessian, a model update $\delta \mathbf{m}$ direction is obtained by solving the following linear system:

$$
\mathbf{H}_k\, \delta \mathbf{m}_k = - \nabla \! \phi_k
$$

We then determine an appropriate step length $\gamma$ and update the model as follows:

$$
\mathbf{m}_{k+1} = \mathbf{m}_k + \gamma \, \delta \mathbf{m}_k
$$

This process is repeated until stopping criteria for the algorithm is reached.


### <span style="color:darkgreen">Exercise (beginner):</span>

Generate an [optimization.InexactGaussNewton](https://docs.simpeg.xyz/content/api/generated/SimPEG.optimization.InexactGaussNewton.html) object to solve the invert problem. Set the following properties for the algorithm:

* maxIter=40
* maxIterLS=20
* maxIterCG=20
* tolCG=1e-3

### Step 9d: Inverse Problem

### <span style="color:darkgreen">Exercise (beginner):</span>

Use the [BaseInvProblem](https://docs.simpeg.xyz/content/api/generated/SimPEG.inverse_problem.BaseInvProblem.html) class to fully define the inverse problem that is solved at each beta (trade-off parameter) iteration. Instantiation requires the following as input arguments:

* data misfit object
* regularization object
* optimization object

## Step 10: Defining Inversion Directives

Directives provide specific instructions to the inversion while it is running. Here, we define SimPEG directives commonly used for least-squares DC resistivitiy inversion.

### 10a: Beta Cooling Directives

<img src="https://raw.githubusercontent.com/simpeg/agrogeo24/main/images/tikhonov_curve.png" width="40%" align="right"/>


### <span style="color:darkgreen">Exercise (beginner):</span>

Define the starting beta, schedule and target misfit directives.

* Generate a [directives.BetaEstimate_ByEig](myst:SimPEG#SimPEG.directives.BetaEstimate_ByEig) object to set the starting beta value. Set the *beta0_ratio* kwarg to 50.
* Generate a [directives.BetaSchedule](myst:SimPEG#SimPEG.directives.BetaSchedule) to set the reduction factor for $\beta$ and the number of Newton iterations at each $\beta$. Set the *coolingFactor* to 2 and *coolingRate* to 2.
* Generate a [directives.TargetMisfit](myst:SimPEG#SimPEG.directives.TargetMisfit) to set stopping criteria for the inversion. Set *chifact* to 1 so inversion terminates when data misfit equals number of data.

### <span style="color:darkorange">Exercise (advanced):</span>

Change the *chifact* to 0.7. This assumes we are worried that we might have over-estimated our uncertainties. In other words, we may end the inversion before the model sufficiently fits the data.

### Other Directives

Other common directives for weighted least-squares inversion of DC resistivity data are:

- [UpdateSensitivityWeights](myst:SimPEG#SimPEG.directives.UpdateSensitivityWeights): apply sensitivity weighting to counteract the natural tendancy of DC resistivity inversion to place materials near the electrodes.

- [UpdatePreconditioner](myst:SimPEG#SimPEG.directives.UpdatePreconditioner): Apply Jacobi preconditioner when solving the optimization problem to reduce the number of conjugate gradient iterations.

- **SaveInversionProgress:** a directive we defined earlier in the notebook to allow us to better Q.C. the inversion result.

In [ ]:
sensitivity_weights = directives.UpdateSensitivityWeights(every_iteration=True)
update_jacobi = directives.UpdatePreconditioner(update_every_iteration=True)
save_inversion = SaveInversionProgress()

The directive objects are organized in a ``list``. The order of the list matters so the directives have been organized for you.

In [ ]:
directives_list = [
    sensitivity_weights,
    update_jacobi,
    starting_beta,
    beta_schedule,
    save_inversion,
    target_misfit,
]

## Step 10: Define and Run the Inversion

### <span style="color:darkgreen">Exercise (beginner):</span>

1. Define the inversion by generating an [inversion.BaseInversion](https://docs.simpeg.xyz/content/api/generated/SimPEG.inversion.BaseInversion.html) object. This object requires the *inv_prob* and *directives_list* as input arguments.
2. Use the *run* method to run the inversion. This method requires the *starting_model* as in input argument.

## Step 11: Inversion Outputs

### 11a: Extract Some Inversion Outputs

In [ ]:
iteration = save_inversion.inversion_results["iteration"]
beta = save_inversion.inversion_results["beta"]
phi_d = save_inversion.inversion_results["phi_d"]
phi_m = save_inversion.inversion_results["phi_m"]
dpred = save_inversion.inversion_results["dpred"]
models = save_inversion.inversion_results["model"]

### 11b: Plot Tikhonov Curve

In [ ]:
fig = plt.figure(figsize=(7, 6))
ax1 = fig.add_subplot(111)
ax2 = ax1.twinx()

ax1.plot(iteration, phi_d, "b-o")
ax1.plot(
    iteration,
    target_misfit.chifact * dc_data.survey.nD * np.ones_like(iteration),
    "k--",
)
ax1.set_xlabel("Iteration", fontsize=20)
ax1.set_ylabel("Data Misfit ($\phi_d$)", color="b", fontsize=18)
ax1.tick_params(axis="y", colors="b")
ax1.set_title("Tikhonov Curve", fontsize=20)

ax2.plot(iteration, phi_m, "r-o")
ax2.tick_params(axis="y", colors="r")
ax2.set_ylabel("Regularization ($\phi_m$)", color="r", fontsize=18)

ax1.set_xlim([np.min(iteration), np.max(iteration)])

plt.show()

### 11c: Choose an Iteration to Further Examine

In [ ]:
model_iteration = -1

### 11d: Plot Observed Data, Prediced Data and Normalized Misfit

In [ ]:
dobs = dc_data.dobs
std = dc_data.standard_deviation

fig = plt.figure(figsize=(7, 9))
data_array = [
    np.abs(dobs),
    np.abs(dpred[model_iteration]),
    (dobs - dpred[model_iteration]) / std,
]
plot_title = ["Observed Data", "Predicted Data", "Normalized Misfit"]
plot_units = ["V/A", "V/A", ""]
scale = ["log", "log", "linear"]
cmap_list = [mpl.cm.viridis, mpl.cm.viridis, mpl.cm.RdYlBu]

ax1 = 3 * [None]
cax1 = 3 * [None]
cbar = 3 * [None]
cplot = 3 * [None]

for ii in range(0, 3):
    ax1[ii] = fig.add_axes([0.15, 0.72 - 0.33 * ii, 0.65, 0.21])
    cax1[ii] = fig.add_axes([0.81, 0.72 - 0.33 * ii, 0.03, 0.21])
    cplot[ii] = plot_pseudosection(
        dc_data.survey,
        data_array[ii],
        "contourf",
        data_locations=True,
        ax=ax1[ii],
        cax=cax1[ii],
        scale=scale[ii],
        cbar_label=plot_units[ii],
        mask_topography=True,
        contourf_opts={"levels": 25, "cmap": cmap_list[ii]},
    )
    ax1[ii].set_title(plot_title[ii])

plt.show()

### 11e: Plot Observed and Predicted Apparent Resistivities

In [ ]:
# Plot apparent conductivity pseudo-section
fig = plt.figure(figsize=(8, 2.75))
ax1 = fig.add_axes([0.1, 0.15, 0.75, 0.78])
plot_pseudosection(
    dc_data.survey,
    dobs=apparent_resistivities,
    plot_type="contourf",
    data_locations=True,
    ax=ax1,
    scale="log",
    cbar_label="$\Omega m$",
    mask_topography=True,
    contourf_opts={"levels": 40, "cmap": mpl.cm.RdYlBu_r},
)
ax1.set_title("Apparent Resistivity (Observed)")
plt.show()

# Plot apparent conductivity pseudo-section
apparent_resistivities_dpred = apparent_resistivity_from_voltage(
    dc_data.survey, dpred[model_iteration]
)

fig = plt.figure(figsize=(8, 2.75))
ax1 = fig.add_axes([0.1, 0.15, 0.75, 0.78])
plot_pseudosection(
    dc_data.survey,
    dobs=apparent_resistivities_dpred,
    plot_type="contourf",
    data_locations=True,
    ax=ax1,
    scale="log",
    cbar_label="$\Omega m$",
    mask_topography=True,
    contourf_opts={"levels": 40, "cmap": mpl.cm.RdYlBu_r},
)
ax1.set_title("Apparent Resistivity (Predicted)")
plt.show()

### 11f: Plot Recovered Model

In [ ]:
air_resistivity = 1e8
background_resistivity = 1e2
sphere_resistivity = 1e1
basement_resistivity = 1e3

In [ ]:
# Define resistivity model
true_resistivity = background_resistivity * np.ones(mesh.nC)

ind_basement = mesh.cell_centers[:, -1] < -16.0
true_resistivity[ind_basement] = basement_resistivity

ind_sphere = model_builder.get_indices_sphere(
    np.r_[-10.0, -10.0], 5.0, mesh.cell_centers
)
true_resistivity[ind_sphere] = sphere_resistivity

true_resistivity = true_resistivity[active_cells]

In [ ]:
# Define a mapping to plot models and ignore inactive cells
plotting_map = maps.InjectActiveCells(mesh, active_cells, np.nan)

plotting_model = [true_resistivity, np.exp(models[model_iteration])]

norm = LogNorm(vmin=1e1, vmax=1e3)

fig = plt.figure(figsize=(9, 6))
ax1 = 2 * [None]
ax2 = 2 * [None]
title_str = [
    "True Resistivity",
    "Recovered Resistivity",
]

for ii in range(0, 2):
    ax1[ii] = fig.add_axes([0.14, 0.55 - 0.5 * ii, 0.68, 0.35])
    mesh.plot_image(
        plotting_map * plotting_model[ii],
        ax=ax1[ii],
        grid=False,
        pcolor_opts={"norm": norm, "cmap": mpl.cm.RdYlBu_r},
    )

    ax1[ii].set_xlim(-75, 75)
    ax1[ii].set_ylim(-50, 0)
    ax1[ii].set_title(title_str[ii])
    ax1[ii].set_xlabel("x (m)")
    ax1[ii].set_ylabel("y (m)")

    ax2[ii] = fig.add_axes([0.84, 0.55 - 0.5 * ii, 0.03, 0.35])
    cbar = mpl.colorbar.ColorbarBase(
        ax2[ii], norm=norm, orientation="vertical", cmap=mpl.cm.RdYlBu_r
    )
    cbar.set_label(r"$\rho (\Omega m)$", rotation=270, labelpad=15, size=12)

pseudo_locations_xy = pseudo_locations(dc_data.survey)
# ax1[1].scatter(pseudo_locations_xy[:, 0], pseudo_locations_xy[:, -1], 0.5, "k")

plt.show()